<a href="https://colab.research.google.com/github/jinyang628/cs4248/blob/load-kaggle/CS4248.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub
import os
import shutil
import pandas as pd
import os

In [ ]:
# Load Kaggle dataset
path = kagglehub.dataset_download("abhi8923shriv/sentiment-analysis-dataset")
print("Kaggle dataset available at:", path)

Using Colab cache for faster access to the 'sentiment-analysis-dataset' dataset.
Kaggle dataset available at: /kaggle/input/sentiment-analysis-dataset


In [ ]:
train_csv_path = os.path.join(path, 'train.csv')
df = pd.read_csv(train_csv_path, encoding='latin1')

display(df.head())

,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105
2,088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18
3,9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26
